<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Embed documents using watsonx's embedding model**


Estimated time needed: **30** minutes


## Overview


Imagine you work in a company that handles a vast amount of text data, including documents, emails, and reports. Your task is to build an intelligent search system that can quickly and accurately retrieve relevant documents based on user queries. Traditional keyword-based search methods often fail to understand the context and semantics of the queries, leading to poor search results.

To address this challenge, you can use embedding models to convert documents into numerical vectors. These vectors capture the semantic meaning of the text, enabling more accurate and context-aware search capabilities. Document embedding is a powerful technique to convert textual data into numerical vectors, which can then be used for various downstream tasks such as search, classification, clustering, and more.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/O_XVADBArH0ck4Wf6uuoBA/embeddings.png" width="60%" alt="embeddings">


In this lab, you will learn how to use embedding models from watsonx.ai and Hugging Face to embed documents. By the end of this lab, you will be able to effectively use these embedding models to transform and utilize textual data in your projects.


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-ribraries">Installing required libraries</a></li>
            <li><a href="#Load-data">Load data</a></li>
            <li><a href="#Split data">Split data</a></li>
        </ol>
    </li>
    <li>
        <a href="#Watsonx-embedding-model">Watsonx embedding model</a>
        <ol>
            <li><a href="#Model-description">Model description</a></li>
            <li><a href="#Build-model">Build model</a></li>
            <li><a href="#Query-embeddings">Query embeddings</a></li>
            <li><a href="#Document-embeddings">Document embeddings</a></li>
        </ol>
    </li>
    <li>
        <a href="#HuggingFace embedding model">HuggingFace embedding models</a>
        <ol>
            <li><a href="#Model-description">Model description</a></li>
            <li><a href="#Build-model">Build model</a></li>
            <li><a href="#Query-embeddings">Query embeddings</a></li>
            <li><a href="#Document-embeddings">Document embeddings</a></li>
        </ol>
    </li>
</ol>

<a href="#Exercises">Exercises</a>
<ol>
    <li><a href="#Exercise-1---Using-another-watsonx-embedding-model">Exercise 1. Using another watsonx embedding model</a></li>
</ol>


## Objectives

After completing this lab, you will be able to:

 - Prepare and preprocess documents for embedding
 - Use watsonx.ai and Hugging Face embedding models to generate embeddings for your documents


----


## Setup


For this lab, you will use the following libraries:

* [`ibm-watson-ai`](https://ibm.github.io/watsonx-ai-python-sdk/fm_embeddings.html#EmbeddingModels) for using embedding models from IBM's watsonx.ai.
* [`langchain`, `langchain-ibm`, `langchain-community`](https://www.langchain.com/) for using relevant features from LangChain.
* [`sentence-transformers`](https://huggingface.co/sentence-transformers) for using embedding models from HuggingFace.


### Installing required libraries

The following required libraries are __not__ preinstalled in the Skills Network Labs environment. __You need to run the following cell__ to install them:

**Note:** The version is being pinned here to specify the version. It's recommended that you do this as well. Even if the library is updated in the future, the installed library could still support this lab work.

This might take around 1-2 minutes.

As `%%capture` is used to capture the installation, you won't see the output process. But after the installation completes, you will see a number beside the cell.


In [1]:
#%%capture
#After executing the cell,please RESTART the kernel and run all the cells.
!pip install --user "ibm-watsonx-ai==1.1.2"
!pip install --user "langchain==0.2.11"
!pip install --user "langchain-ibm==0.1.11"
!pip install --user "langchain-community==0.2.10"
!pip install --user "sentence-transformers==3.0.1"
print("done")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.6/966.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.5 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.13.6-py3-none-any.whl size=77230 sha256=6f00b7f40304341252399226f5565c4f4a4d262195b4775363b08eb2bfad640f
  Stored in directory: /root/.cache/pip/wheels/17/8c/76/f9472a53a2a80da414dd53c

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 22.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/88

After you install the libraries, restart your kernel. You can do that by clicking the **Restart the kernel** icon.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/build-a-hotdog-not-hotdog-classifier-guided-project/images/Restarting_the_Kernel.png" width="50%" alt="Restart kernel">


## Load data


A text file has been prepared as the source document for the downstream embedding task.

Now, let's download and load it using LangChain's `TextLoader`.


In [1]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/i5V3ACEyz6hnYpVq6MTSvg/state-of-the-union.txt"

--2025-06-21 09:33:34--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/i5V3ACEyz6hnYpVq6MTSvg/state-of-the-union.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘state-of-the-union.txt’

state-of-the-union. 100%[===================>]  38.11K  --.-KB/s    in 0.06s   

2025-06-21 09:33:35 (607 KB/s) - ‘state-of-the-union.txt’ saved [39027/39027]



In [2]:
from langchain_community.document_loaders import TextLoader

In [3]:
loader = TextLoader("state-of-the-union.txt")
data = loader.load()

Let's take a look at the document.


In [4]:
data

[Document(metadata={'source': 'state-of-the-union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determin

## Split data


Since the embedding model has a maximum input token limit, you cannot input the entire document at once. Instead, you need to split it into chunks.

The following code shows how to use LangChain's `RecursiveCharacterTextSplitter` to split the document into chunks.
- Use the default separator list, which is `["\n\n", "\n", " ", ""]`.
- Chunk size is set to `100`. This should be set to less than the model's maximum input token.
- Chunk overlap is set to `20`.
- The length function is `len`.


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

In [7]:
chunks = text_splitter.split_text(data[0].page_content)

Let's see how many chunks you get.


In [8]:
len(chunks)

570

Let's also see what these chunks looks like.


In [9]:
chunks

['Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and',
 'of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.',
 'Last year COVID-19 kept us apart. This year we are finally together again.',
 'Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.',
 'With a duty to one another to the American people to the Constitution.',
 'And with an unwavering resolve that freedom will always triumph over tyranny.',
 'Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he',
 'world thinking he could make it bend to his menacing ways. But he badly miscalculated.',
 'He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of',
 'he met a wall of strength he never imagined.',
 'He met the Ukrainian people.',
 'From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their',
 'courage, th

## Watsonx embedding model


### Model description


In this section, you will use IBM `slate-125m-english-rtrvr` model as an example embedding model.

The slate.125m.english.rtrvr model is a [standard sentence](https://www.sbert.net/) transformers model based on bi-encoders. The model produces an embedding for a given input, e.g., query, passage, document, etc. At a high level, the model is trained to maximize the cosine similarity between two input pieces of text, e.g., text A (query text) and text B (passage text), which results in the sentence embeddings q and p.These sentence embeddings can be compared using cosine similarity, which measures the distance between sentences by calculating the distance between their embeddings.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/NDCHhZfcC96jggb2hMdJhg/fm-slate-125m-english-rtrvr-cosine.jpg" width="50%">


The embedding model, `slate.125m.english` formerly known as WatBERT, has the same architecture as a RoBERTa base transformer model and has ~125 million parameters and an embedding dimension of `768`.


|Model name|API model_id|Maximum input tokens|Number of dimensions|More information|
|-|-|-|-|-|
|slate-125m-english-rtrvr|ibm/slate-125m-english-rtrvr|512|768|[model card](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-slate-125m-english-rtrvr-model-card.html?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-Embed+documents+with+watsonx%E2%80%99s+embedding_v1_1721662184&context=wx)|


### Build model


The following code shows how to build the `slate-125m-english-rtrvr` model from IBM watsonx.ai API.


First, import the necessary dependencies.
- `WatsonxEmbeddings` is a class/dependence that can be used to form an embedding model object.
- `EmbedTextParamsMetaNames` is a dependence that controls the embedding parameters.


In [10]:
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

In [21]:
import os
from getpass import getpass
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

try:
    from google.colab import userdata
#    watsonx_api_key = userdata.get('WATSONX_APIKEY')
 # In Google Colab, use the built-in secrets feature
   watsonx_api_key = userdata.get('IBM_API_KEY')
    ibm_project_id = userdata.get('IBM_PROJECT_ID')
except:
    watsonx_api_key = getpass("Enter your WATSONX_APIKEY: ")

# Configure embedding parameters
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

print(watsonx_api_key)

# Create WatsonxEmbeddings with API key
watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id=ibm_project_id,
    apikey=watsonx_api_key,  # Add the API key here
    params=embed_params,
)

print("✓ WatsonxEmbeddings configured successfully")

am7HHaQuCo5s3jC39twjFp5U2VihaRi_fKpPc2UbH_8-
✓ WatsonxEmbeddings configured successfully


### Query embeddings


Now, create an embedding based on a single sentence, which can be treated as a query.


Use the `embed_query` method.


In [22]:
query = "How are you?"

query_result = watsonx_embedding.embed_query(query)

Let's see the length/dimension of this embedding.


In [23]:
len(query_result)

768

It has a dimension of `768`, which aligns with the model description.


Next, take a look at the first five results from the embeddings.


In [24]:
query_result[:5]

[-0.06722454, -0.023729993, 0.017487843, -0.013195328, -0.039584607]

### Document embeddings


After creating the query embeddings, you will be guided on how to create embeddings from documents, which are a list a text chunks.


Use `embed_documents`. The parameter `chunks` should be a list of text. Here, chunks is a list of documents you get from before after splitting the whole document.


In [25]:
doc_result = watsonx_embedding.embed_documents(chunks)

As each piece of text is embedded into a vector, so the length of the `doc_result` should be the same as the length of chunks.


In [26]:
len(doc_result)

570

Now, take a look at the first five results from the embeddings of the first piece of text.


In [27]:
doc_result[0][:5]

[-0.009708214, -0.0016381955, -0.01340118, -0.005656508, -0.031278424]

In [29]:
doc_result[1][:5]

[-0.04085868, 0.042270757, -0.043085914, 0.029131081, 0.0031788636]

Check the embedding dimension to see if it is also 768.


In [28]:
len(doc_result[0])

768

## Hugging Face embedding model - (Garbage - skipped !!!)


### Model description


In this section, you will use the `all-mpnet-base-v2` from HuggingFace as an example embedding model.

It is a sentence-transformers model. It maps sentences and paragraphs to a 768-dimensional dense vector space and can be used for tasks like clustering or semantic search. It used the pre-trained `Microsoft/money-base` model and fine-tuned it on a 1B sentence pairs dataset. For more information, please refer to [here](https://huggingface.co/sentence-transformers/all-mpnet-base-v2).


### Build model


To build the model, you need to import the `HuggingFaceEmbeddings` dependence first.


In [30]:
from langchain_community.embeddings import HuggingFaceEmbeddings

Then, you specify the model name.


In [31]:
model_name = "sentence-transformers/all-mpnet-base-v2"

Here we create a embedding model object.


In [2]:
# First, install the required packages
!pip install langchain-huggingface sentence-transformers
!pip install langchain-community
!pip install sentence-transformers

In [1]:
#huggingface_embedding = HuggingFaceEmbeddings(model_name=model_name)

# Updated import (new location)
from langchain_huggingface import HuggingFaceEmbeddings

# Updated code
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # or your preferred model
huggingface_embedding = HuggingFaceEmbeddings(model_name=model_name)

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

### Query embeddings


Let's create the embeddings from the same sentence, but using the Hugging Face embedding model.


In [ ]:
query = "How are you?"

In [ ]:
query_result = huggingface_embedding.embed_query(query)

In [ ]:
query_result[:5]

Do you see the differences between embeddings that are created by the watsonx embedding model and the Hugging Face embedding model?


### Document embeddings


Next, you can do the same for creating embeddings from documents.


In [ ]:
doc_result = huggingface_embedding.embed_documents(chunks)
doc_result[0][:5]

In [ ]:
len(doc_result[0])

# Exercises


### Exercise 1 - Using another watsonx embedding model
Watsonx.ai also supports other embedding models, for which you can find more information [here](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-slate-30m-english-rtrvr-model-card.html?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-Embed+documents+with+watsonx%E2%80%99s+embedding_v1_1721662184&context=wx). Can you try to use another embedding model to create embeddings for the document?


In [2]:

# Step 1: Uninstall conflicting packages
!pip uninstall pydantic langchain langchain-core langchain-ibm -y

# Step 2: Install specific compatible versions
!pip install pydantic==1.10.12
!pip install langchain==0.1.0
!pip install langchain-ibm==0.1.0
# Install/update IBM watsonx package
!pip install ibm-watsonx-ai --upgrade --quiet
print("done")

Found existing installation: pydantic 1.10.12
Uninstalling pydantic-1.10.12:
  Successfully uninstalled pydantic-1.10.12
Found existing installation: langchain 0.1.0
Uninstalling langchain-0.1.0:
  Successfully uninstalled langchain-0.1.0
Found existing installation: langchain-core 0.1.53
Uninstalling langchain-core-0.1.53:
  Successfully uninstalled langchain-core-0.1.53
Found existing installation: langchain-ibm 0.1.0
Uninstalling langchain-ibm-0.1.0:
  Successfully uninstalled langchain-ibm-0.1.0
  Using cached pydantic-1.10.12-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (149 kB)
Using cached pydantic-1.10.12-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-settings 2.9.1 requires pydantic>=2.7.0, but you have pydantic 1.10.12 which is incompatible.
albumentat

In [1]:
# First, downgrade Pydantic to v1
#!pip install "pydantic<2.0.0"

# Restart your runtime after this, then run your code
import os
from google.colab import userdata

# Set the three secret keys in OS environment before imports
os.environ['IBM_API_KEY'] = userdata.get('IBM_API_KEY')
os.environ['WATSONX_APIKEY'] = userdata.get('WATSONX_APIKEY')
os.environ['IBM_PROJECT_ID'] = userdata.get('IBM_PROJECT_ID')

from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

# Get the values from environment
ibm_api_key = os.environ.get('IBM_API_KEY')
ibm_project_id = os.environ.get('IBM_PROJECT_ID')

embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-30m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id=ibm_project_id,
    apikey=ibm_api_key,
    params=embed_params,
)

doc_result = watsonx_embedding.embed_documents(chunks)
print(doc_result[0][:5])

ImportError: cannot import name 'EmbedTextParamsMetaNames' from 'ibm_watsonx_ai.metanames' (/usr/local/lib/python3.11/dist-packages/ibm_watsonx_ai/metanames.py)

In [3]:
import os
from google.colab import userdata

# Set environment variables with validation
def set_environment_variables():
    """Set and validate environment variables from Colab secrets"""
    required_keys = ['IBM_API_KEY', 'WATSONX_APIKEY', 'IBM_PROJECT_ID']

    for key in required_keys:
        try:
            value = userdata.get(key)
            if value:
                os.environ[key] = value
                print(f"✅ {key} loaded from Colab secrets")
            else:
                print(f"❌ {key} is empty in Colab secrets")
                return False
        except Exception as e:
            print(f"❌ Error loading {key}: {e}")
            return False
    return True

# Set environment variables
if not set_environment_variables():
    raise ValueError("Failed to load required secrets from Colab userdata")

# Install/update IBM watsonx package
!pip install ibm-watsonx-ai --upgrade --quiet

try:
    # Import IBM watsonx components with correct paths
    from ibm_watsonx_ai import APIClient, Credentials

    # Try different import paths for EmbeddingModel
    try:
        from ibm_watsonx_ai.foundation_models import Embeddings
        print("✅ Using Embeddings class")
        embedding_class = Embeddings
    except ImportError:
        try:
            from ibm_watsonx_ai.foundation_models.embeddings import Embeddings
            print("✅ Using embeddings.Embeddings class")
            embedding_class = Embeddings
        except ImportError:
            try:
                # Alternative: use the generate method directly
                from ibm_watsonx_ai.foundation_models import generate_embeddings
                print("✅ Using generate_embeddings function")
                embedding_class = None
            except ImportError:
                print("❌ Could not find embedding functionality")
                # Let's explore what's available
                import ibm_watsonx_ai.foundation_models as fm
                print("Available in foundation_models:")
                print([attr for attr in dir(fm) if not attr.startswith('_')])
                raise ImportError("Could not find embedding functionality")

    print("✅ IBM watsonx packages imported successfully")

except ImportError as e:
    print(f"❌ Import error: {e}")
    raise

# Get credentials with validation
ibm_api_key = os.environ.get('IBM_API_KEY')
ibm_project_id = os.environ.get('IBM_PROJECT_ID')

if not ibm_api_key or not ibm_project_id:
    raise ValueError("Missing required IBM_API_KEY or IBM_PROJECT_ID")

print(f"✅ Using Project ID: {ibm_project_id[:8]}...")

try:
    # Create credentials and client
    credentials = Credentials(
        url="https://us-south.ml.cloud.ibm.com",
        api_key=ibm_api_key
    )
    print("✅ Credentials created")

    client = APIClient(credentials)
    client.set.default_project(ibm_project_id)
    print("✅ API Client configured")

    # Create embedding model using the correct class/method
    if embedding_class:
        embedding_model = embedding_class(
            model_id="ibm/slate-30m-english-rtrvr",
            credentials=credentials,
            project_id=ibm_project_id
        )
        print("✅ Embedding model created")
    else:
        # Use direct API calls if class is not available
        print("✅ Will use direct API calls for embeddings")

except Exception as e:
    print(f"❌ Error setting up watsonx components: {e}")
    raise

# Create embedding functions that work with the available API
def embed_documents(texts):
    """
    Embed documents using IBM watsonx with fallback methods
    """
    if not texts:
        raise ValueError("Input texts cannot be empty")

    if not isinstance(texts, list):
        texts = [texts]

    try:
        print(f"📝 Embedding {len(texts)} text(s)...")

        if embedding_class and hasattr(embedding_model, 'embed_documents'):
            # Method 1: Use EmbeddingModel.embed_documents
            result = embedding_model.embed_documents(texts)
        elif embedding_class and hasattr(embedding_model, 'generate'):
            # Method 2: Use generate method
            result = embedding_model.generate(inputs=texts)
            # Extract embeddings from result
            if hasattr(result, 'results'):
                result = [item['embedding'] for item in result.results]
        else:
            # Method 3: Direct API call
            result = call_embedding_api_directly(texts)

        print(f"✅ Successfully created {len(result)} embeddings")
        return result

    except Exception as e:
        print(f"❌ Embedding failed: {e}")
        # Try fallback method
        print("🔄 Trying direct API call...")
        return call_embedding_api_directly(texts)

def call_embedding_api_directly(texts):
    """Fallback method using direct API calls"""
    import requests
    import json

    # Get access token
    def get_access_token():
        url = "https://iam.cloud.ibm.com/identity/token"
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = f"grant_type=urn:iam:params:oauth:grant-type:apikey&apikey={ibm_api_key}"

        response = requests.post(url, headers=headers, data=data)
        if response.status_code == 200:
            return response.json()["access_token"]
        else:
            raise Exception(f"Failed to get token: {response.text}")

    # Get token and make API call
    token = get_access_token()

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/embeddings"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "model_id": "ibm/slate-30m-english-rtrvr",
        "project_id": ibm_project_id,
        "inputs": texts
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        embeddings = [item["embedding"] for item in result["results"]]
        return embeddings
    else:
        raise Exception(f"API call failed: {response.text}")

def embed_query(text):
    """Embed a single query text"""
    if not text:
        raise ValueError("Query text cannot be empty")

    result = embed_documents([text])
    return result[0] if result else None

print("✅ IBM watsonx embeddings ready!")
print("\nAvailable functions:")
print("  - embed_documents(texts): Embed a list of texts")
print("  - embed_query(text): Embed a single query")

# Test with sample data
print("\n🧪 Running test with sample data...")
test_chunks = [
    "LangChain is a framework for building applications with language models.",
    "IBM watsonx.ai provides foundation models for enterprise AI applications."
]

try:
    doc_result = embed_documents(test_chunks)
    print(f"✅ Test successful!")
    print(f"   - Embedded {len(test_chunks)} test chunks")
    print(f"   - Each embedding has {len(doc_result[0])} dimensions")
    print(f"   - First embedding preview: {doc_result[0][:5]}")

    # Test single query
    query_result = embed_query("What is machine learning?")
    print(f"   - Query embedding dimensions: {len(query_result)}")
    print(f"   - Query embedding preview: {query_result[:5]}")

    # Make it available for your use
    if 'chunks' in locals() and chunks:
        print(f"\n📋 Processing your {len(chunks)} chunks...")
        doc_result = embed_documents(chunks)
        print(f"✅ Your chunks embedded successfully!")

except Exception as e:
    print(f"❌ Test failed: {e}")
    print("\nTrying alternative installation...")
    !pip install ibm-watsonx-ai==1.0.* --force-reinstall --quiet
    print("Please restart runtime and try again")

print("\n💡 Usage:")
print("   doc_result = embed_documents(['text1', 'text2'])")
print("   query_embedding = embed_query('search query')")

✅ IBM_API_KEY loaded from Colab secrets
✅ WATSONX_APIKEY loaded from Colab secrets
✅ IBM_PROJECT_ID loaded from Colab secrets
✅ Using Embeddings class
✅ IBM watsonx packages imported successfully
✅ Using Project ID: 839fdc16...
✅ Credentials created
✅ API Client configured
✅ Embedding model created
✅ IBM watsonx embeddings ready!

Available functions:
  - embed_documents(texts): Embed a list of texts
  - embed_query(text): Embed a single query

🧪 Running test with sample data...
📝 Embedding 2 text(s)...
✅ Successfully created 2 embeddings
✅ Test successful!
   - Embedded 2 test chunks
   - Each embedding has 384 dimensions
   - First embedding preview: [-0.029391885, 0.01597066, 0.012774168, 0.019821336, -0.013619158]
📝 Embedding 1 text(s)...
✅ Successfully created 1 embeddings
   - Query embedding dimensions: 384
   - Query embedding preview: [0.0061062225, 0.04559856, 0.018575016, 0.0128906, 0.027421473]

💡 Usage:
   doc_result = embed_documents(['text1', 'text2'])
   query_embedding

<details>
    <summary>Click here for Solution</summary>

```python

from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-30m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    params=embed_params,
)

doc_result = watsonx_embedding.embed_documents(chunks)

doc_result[0][:5]

```

</details>


## Authors


[Kang Wang](https://author.skills.network/instructors/kang_wang)

Kang Wang is a Data Scientist in IBM. He is also a PhD Candidate in the University of Waterloo.


### Other Contributors


[Joseph Santarcangelo](https://author.skills.network/instructors/joseph_santarcangelo)

Joseph has a Ph.D. in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

[Hailey Quach](https://author.skills.network/instructors/hailey_quach)

Hailey is a Data Scientist Intern at IBM. She is also pursuing a BSc in Computer Science, Honors at Concordia University, Montreal.


```{## Change Log}
```


```{|Date (YYYY-MM-DD)|Version|Changed By|Change Description||-|-|-|-||2024-07-22|0.1|Kang Wang|Create the lab|}
```


Copyright © IBM Corporation. All rights reserved.

